In [2]:
%run "../00_project_config.ipynb"
%run "10_data_objects.ipynb"

import pandas as pd
import pathlib as pl
import sys

data_status_annotation = PROJECT_BASE.joinpath(
    "annotations", "projectmng", "data_production_status.tsv"
)

data_status_df = pd.read_csv(data_status_annotation, sep="\t", header=0, comment="#")

DATA_STATUS = dict()
for row in data_status_df.itertuples(index=False):
    if row.sample == "GM19320":
        sys.stderr.write("\nSkipping over 19320 - not enough data\n")
        continue
    if row.hifi_complete == "skip" or row.ont_complete == "skip" or row.igsr_status == "skip":
        continue
    if row.hifi_complete == "no" or row.ont_complete == "no":
        assert row.igsr_status == "incomplete", row
    if row.hifi_complete == "yes" and row.ont_complete == "yes":
        assert row.igsr_status == "complete"
    sample_name = row.sample
    if sample_name.startswith("GM"):
        sample_name = "NA" + sample_name[2:]
    sample_info = {
        "order_num": int(row.order_num),
        "sex": row.sex.lower(),
        "batch_number": int(row.sample_batch),
        "is_pilot_sample": row.is_pilot,
        "hifi_complete": True if row.hifi_complete == "yes" else False,
        "ont_complete": True if row.ont_complete == "yes" else False
    }
    DATA_STATUS[sample_name] = sample_info

SAMPLES = dict()
for row in HGSVC_SAMPLES.itertuples(index=False):
    if row.sample == "GM19320":
        sys.stderr.write("\nSkipping over 19320 - not enough data\n")
        continue
    sample = Sample(row.sample, row.order_num, row.sex)
    try:
        sample_data_status = DATA_STATUS[sample.name]
        assert sample_data_status["order_num"] == sample.hgsvc_num
        assert sample_data_status["sex"] == sample.sex
        sample.hifi_complete = sample_data_status["hifi_complete"]
        sample.ont_complete = sample_data_status["ont_complete"]
        sample.batch_num = sample_data_status["batch_number"]
    except KeyError:
        pass
    SAMPLES[sample.name] = sample
    SAMPLES[sample.alt] = sample


ADD_SAMPLES = [
    ("NA24143", 100, "female"),
    ("NA24149", 101, "male"),
    ("NA12878", 102, "female")
]

for sample_name, num, sex in ADD_SAMPLES:
    sample = Sample(sample_name, num, sex)
    SAMPLES[sample.name] = sample
    SAMPLES[sample.alt] = sample
    



Skipping over 19320 - not enough data
